In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


**1-QADAM BERILGAN DATASETLARDA QANDAY MA'LUMOTLAR BORLIGINI KO"RIB OLAMIZ**


In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


from sklearn import metrics







**Ma'lumotlarni o'qib olamiz**

In [8]:
train_data = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv")
train_data.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [9]:
test_data = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv")

In [54]:
submission = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv")
submission.head()

,id,price
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [11]:
df_train = train_data

In [12]:
df_train.isnull().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [13]:
df_train.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [24]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [25]:
x = df_train.drop("price" ,axis=1)
y = df_train['price']

In [27]:

nums = ['duration', 'days_left']
cats = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']


num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, nums),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cats)
])

predict_train = full_pipeline.fit_transform(x)


In [28]:
predict_train.toarray()

array([[ 0.28952803, -0.36141789,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.45682723, -1.38897606,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.36291029, -1.53577008,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 3.23458153, -0.65500594,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.58012154, -0.36141789,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.84836171,  0.00556716,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [29]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()
LR_model.fit(predict_train, y)

LinearRegression()

In [37]:
predictions_train = LR_model.predict(predict_train)

y_true = np.asarray(y).ravel()

mae_train = mean_absolute_error(y_true, predictions_train)
print("Mean Absolute Error on Training Set:", mae_train)

#Mean Squared Error (MSE)
mse_train = mean_squared_error(y_true, predictions_train)
print("Mean Squared Error on Training Set:", mse_train)

# Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
print("Root Mean Squared Error on Training Set:", rmse_train)

# R-squared (R2) Score
r2_train = r2_score(y_true, predictions_train)
print("R-squared Score on Training Set:", r2_train)


Mean Absolute Error on Training Set: 4144.925468226864
Mean Squared Error on Training Set: 36730618.297167726
Root Mean Squared Error on Training Set: 6060.579039759133
R-squared Score on Training Set: 0.9291865656620019


In [38]:
from sklearn.svm import SVR

svm_model = SVR()
svm_model.fit(predict_train, y_true)

predictions_svm = svm_model.predict(predict_train)

mae_svm = mean_absolute_error(y_true, predictions_svm)
mse_svm = mean_squared_error(y_true, predictions_svm)
rmse_svm = np.sqrt(mse_svm)
r2_svm = r2_score(y_true, predictions_svm)

print("SVM Regression Results:")
print("Mean Absolute Error:", mae_svm)
print("Mean Squared Error:", mse_svm)
print("Root Mean Squared Error:", rmse_svm)
print("R-squared Score:", r2_svm)


SVM Regression Results:
Mean Absolute Error: 15865.02439867996
Mean Squared Error: 681422700.183343
Root Mean Squared Error: 26104.074398134537
R-squared Score: -0.31372364182541856


In [39]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor()
tree_model.fit(predict_train, y_true)

predictions_tree = tree_model.predict(predict_train)

mae_tree = mean_absolute_error(y_true, predictions_tree)
mse_tree = mean_squared_error(y_true, predictions_tree)
rmse_tree = np.sqrt(mse_tree)
r2_tree = r2_score(y_true, predictions_tree)

print("Decision Tree Regression Results:")
print("Mean Absolute Error:", mae_tree)
print("Mean Squared Error:", mse_tree)
print("Root Mean Squared Error:", rmse_tree)
print("R-squared Score:", r2_tree)


Decision Tree Regression Results:
Mean Absolute Error: 0.8832
Mean Squared Error: 1775.75995
Root Mean Squared Error: 42.13976684795491
R-squared Score: 0.9999965764894126


In [40]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor()
forest_model.fit(predict_train, y_true)

predictions_forest = forest_model.predict(predict_train)

mae_forest = mean_absolute_error(y_true, predictions_forest)
mse_forest = mean_squared_error(y_true, predictions_forest)
rmse_forest = np.sqrt(mse_forest)
r2_forest = r2_score(y_true, predictions_forest)

print("Random Forest Regression Results:")
print("Mean Absolute Error:", mae_forest)
print("Mean Squared Error:", mse_forest)
print("Root Mean Squared Error:", rmse_forest)
print("R-squared Score:", r2_forest)


Random Forest Regression Results:
Mean Absolute Error: 564.6043044154762
Mean Squared Error: 1523886.272070998
Root Mean Squared Error: 1234.4578859041721
R-squared Score: 0.9970620799357958


In [45]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()
xgb_model.fit(predict_train, y_true)

predictions_xgb = xgb_model.predict(predict_train)

mae_xgb = mean_absolute_error(y_true, predictions_xgb)
mse_xgb = mean_squared_error(y_true, predictions_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_true, predictions_xgb)

print("XGBoost Regression Results:")
print("Mean Absolute Error:", mae_xgb)
print("Mean Squared Error:", mse_xgb)
print("Root Mean Squared Error:", rmse_xgb)
print("R-squared Score:", r2_xgb)


XGBoost Regression Results:
Mean Absolute Error: 1880.3718255218505
Mean Squared Error: 10234035.931664556
Root Mean Squared Error: 3199.0679785938523
R-squared Score: 0.9802696696909259


In [46]:
test_predict = full_pipeline.transform(test_data)

In [48]:
test_predict2 = xgb_model.predict(test_predict)

In [49]:
print(test_predict2)

[56573.168 59782.5   22999.057 ... 52584.58  48445.492 63013.78 ]


In [52]:
sample = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv")
sample.head()

,id,price
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [57]:
sample['price'] = test_predict2
sample['price'] = sample['price'].apply(lambda x: "${:,.2f}".format(x))
sample.head()

,id,price
0,1,"$56,573.17"
1,2,"$59,782.50"
2,3,"$22,999.06"
3,4,"$3,086.57"
4,5,"$6,136.48"


In [59]:
sample.to_csv('predict.csv', index=False)

**Xulosa**

**XGBoost**: Ushbu dataset uchun XGBoost modeli ham yaxshi ishladi, past xato va yuqori R-kvadrat natijalarini ko'rsatdi. Linear Regression overfitting tufayli olinmadi